## PySpark Code Snippets

### UPSERT

In [ ]:
# check if table exists
if (updates.count() == 0):
    varRecordsInserted = 0
    varRecordsUpdated = 0
elif (DeltaTable.isDeltaTable(spark, tablePath)):
    # get handle on table
    deltaAce = DeltaTable.forPath(spark, tablePath)
    # UPSERT logic
    deltaAce.alias("source").merge(
        updates.alias("updates"), qry
    ).whenMatchedUpdateAll(
    ).whenNotMatchedInsertAll(
    ).execute()
    display(deltaAce.history())
    operations = deltaAce.history()
    lastOperation = operations.filter(operations['operation'] == 'MERGE').first()
    display(lastOperation)
    varTotalRows = int(lastOperation.operationMetrics["numSourceRows"])
    varRecordsInserted = int(lastOperation.operationMetrics["numTargetRowsInserted"])
    varRecordsUpdated = int(lastOperation.operationMetrics["numTargetRowsUpdated"])
#if table doesn't exist create it from dataframe
else:
    tableName = f'lhBronze.{varBronzeTbl}'
    updates.write.format("delta").mode("overwrite").saveAsTable(tableName)
    varTotalRows = updates.count()
    varRecordsInserted = updates.count()
    varRecordsUpdated = 0

# Results
results = {
    'Inserted' : varRecordsInserted,
    'Updated' : varRecordsUpdated
}
mssparkutils.notebook.exit(results)